In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Importing the dependencies

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import sqlite3
import csv
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from wordcloud import WordCloud
import re
import os
from sqlalchemy import create_engine # database connection
import datetime as dt
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
from sklearn.metrics import f1_score,precision_score,recall_score
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from skmultilearn.adapt import mlknn
from skmultilearn.problem_transform import ClassifierChain
from skmultilearn.problem_transform import BinaryRelevance
from skmultilearn.problem_transform import LabelPowerset
from sklearn.naive_bayes import GaussianNB
from datetime import datetime

<h1> 1. Exploratory Data Analysis </h1>

<h2> 1.1 Data Loading and Cleaning </h2>

<h3>1.1.1 Using Pandas with SQLite to Load the data</h3>

In [ ]:
if not os.path.isfile('train.db'):
    start = datetime.now()
    disk_engine = create_engine('sqlite:///train.db')
    start = dt.datetime.now()
    chunksize = 180000
    j = 0
    index_start = 1
    for df in pd.read_csv('/kaggle/input/facebook-recruiting-iii-keyword-extraction/Train.zip', names=['Id', 'Title', 'Body', 'Tags'], chunksize=chunksize, iterator=True, encoding='utf-8', ):
        df.index += index_start
        j+=1
        print('{} rows'.format(j*chunksize))
        df.to_sql('data', disk_engine, if_exists='append')
        index_start = df.index[-1] + 1
    print("Time taken to run this cell :", datetime.now() - start)

<h3> 1.1.2 Counting the number of rows </h3>

In [ ]:
if os.path.isfile('train.db'):
    start = datetime.now()
    con = sqlite3.connect('train.db')
    num_rows = pd.read_sql_query("""SELECT count(*) FROM data""", con)
    #Always remember to close the database
    print("Number of rows in the database :","\n",num_rows['count(*)'].values[0])
    con.close()
    print("Time taken to count the number of rows :", datetime.now() - start)
else:
    print("Please download the train.db file from drive or run the above cell to genarate train.db file")

<h3>1.1.3 Checking for duplicates </h3>

In [ ]:
if os.path.isfile('train.db'):
    start = datetime.now()
    con = sqlite3.connect('train.db')
    df_no_dup = pd.read_sql_query('SELECT Title, Body, Tags, COUNT(*) as cnt_dup FROM data GROUP BY Title, Body, Tags', con)
    con.close()
    print("Time taken to run this cell :", datetime.now() - start)
else:
    print("Please download the train.db file from drive or run the first to genarate train.db file")

In [ ]:
df_no_dup.head()

In [ ]:
print("number of duplicate questions :", num_rows['count(*)'].values[0]- df_no_dup.shape[0])

In [ ]:
# number of times each question appeared in our database
df_no_dup.cnt_dup.value_counts()

In [ ]:
start = datetime.now()
# Check for and handle None values in the 'Tags' column
df_no_dup['Tags'] = df_no_dup['Tags'].fillna('')  # Replace None with an empty string
df_no_dup["tag_count"] = df_no_dup["Tags"].apply(lambda text: len(text.split(" ")))
# adding a new feature number of tags per question
print("Time taken to run this cell :", datetime.now() - start)
df_no_dup.head()

In [ ]:
# distribution of number of tags per question
df_no_dup.tag_count.value_counts()

In [ ]:
#Creating a new database with no duplicates
if not os.path.isfile('train_no_dup.db'):
    disk_dup = create_engine("sqlite:///train_no_dup.db")
    no_dup = pd.DataFrame(df_no_dup, columns=['Title', 'Body', 'Tags'])
    no_dup.to_sql('no_dup_train',disk_dup)

In [ ]:
#This method seems more appropriate to work with this much data.
#creating the connection with database file.
if os.path.isfile('train_no_dup.db'):
    start = datetime.now()
    con = sqlite3.connect('train_no_dup.db')
    tag_data = pd.read_sql_query("""SELECT Tags FROM no_dup_train""", con)
    #Always remember to close the database
    con.close()

    # Let's now drop unwanted column.
    tag_data.drop(tag_data.index[0], inplace=True)
    #Printing first 5 columns from our data frame
    tag_data.head()
    print("Time taken to run this cell :", datetime.now() - start)
else:
    print("Please download the train.db file from drive or run the above cells to genarate train.db file")

<h2> 2.2 Analysis of Tags </h2>

<h3> 2.2.1 Total number of unique tags </h3>

In [ ]:
# Importing & Initializing the "CountVectorizer" object, which 
#is scikit-learn's bag of words tool.


vectorizer = CountVectorizer(tokenizer = lambda x: x.split())
# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of strings.
tag_dtm = vectorizer.fit_transform(tag_data['Tags'])

In [ ]:
print('Number of data points :', tag_dtm.shape[0])
print('Number of unique tags :', tag_dtm.shape[1])

In [ ]:
tags=vectorizer.get_feature_names_out()
print(tags[:10])

<h3> 2.2.3 Number of times a tag appeared </h3>

In [ ]:
freq=tag_dtm.sum(axis=0).A1
result=dict(zip(tags,freq))

In [ ]:
if not os.path.isfile('tag_counts_dict_dtm.csv'):
    with open('tag_counts_dict_dtm.csv','w') as csv_file:
        writer=csv.writer(csv_file)
        for key, value in result.items():
            writer.writerow([key,value])
tag_df=pd.read_csv('tag_counts_dict_dtm.csv',names=['Tags','Counts'])
tag_df.head()

In [ ]:
tag_df_sorted=tag_df.sort_values(['Counts'],ascending=False)
tag_counts=tag_df_sorted['Counts'].values

In [ ]:
plt.plot(tag_counts)
plt.title("Distribution of number of times tag appeared questions")
plt.grid()
plt.xlabel("Tag number")
plt.ylabel("Number of times tag appeared")
plt.show()

In [ ]:
plt.plot(tag_counts[0:10000])
plt.title('first 10k tags: Distribution of number of times tag appeared questions')
plt.grid()
plt.xlabel("Tag number")
plt.ylabel("Number of times tag appeared")
plt.show()
print(len(tag_counts[0:10000:25]), tag_counts[0:10000:25])

In [ ]:
plt.plot(tag_counts[0:1000])
plt.title('first 1k tags: Distribution of number of times tag appeared questions')
plt.grid()
plt.xlabel("Tag number")
plt.ylabel("Number of times tag appeared")
plt.show()
print(len(tag_counts[0:1000:5]), tag_counts[0:1000:5])

In [ ]:
plt.plot(tag_counts[0:500])
plt.title('first 500 tags: Distribution of number of times tag appeared questions')
plt.grid()
plt.xlabel("Tag number")
plt.ylabel("Number of times tag appeared")
plt.show()
print(len(tag_counts[0:500:5]), tag_counts[0:500:5])

In [ ]:
plt.plot(tag_counts[0:100], c='b')
plt.scatter(x=list(range(0,100,5)), y=tag_counts[0:100:5], c='orange', label="quantiles with 0.05 intervals")
# quantiles with 0.25 difference
plt.scatter(x=list(range(0,100,25)), y=tag_counts[0:100:25], c='m', label = "quantiles with 0.25 intervals")

for x,y in zip(list(range(0,100,25)), tag_counts[0:100:25]):
    plt.annotate("({} , {})".format(x,y), xy=(x,y), xytext=(x-0.05, y+500))

plt.title('first 100 tags: Distribution of number of times tag appeared questions')
plt.grid()
plt.xlabel("Tag number")
plt.ylabel("Number of times tag appeared")
plt.legend()
plt.show()
print(len(tag_counts[0:100:5]), tag_counts[0:100:5])

In [ ]:
#store the tags greater than 10k in one list
lst_tag_gt_10k=tag_df[tag_df.Counts>10000].Tags
#Print the length of the list
print('{} tags more than 10 k'.format(len(lst_tag_gt_10k)))
# Store tags greater than 100K in one list
lst_tags_gt_100k = tag_df[tag_df.Counts>100000].Tags
#Print the length of the list.
print ('{} Tags are used more than 100000 times'.format(len(lst_tags_gt_100k)))

<b>Observations:</b><br />
1. There are total 153 tags which are used more than 10000 times.
2. 14 tags are used more than 100000 times.
3. Most frequent tag (i.e. c#) is used 331505 times.
4. Since some tags occur much more frequenctly than others, Micro-averaged F1-score is the appropriate metric for this probelm.

<h3> 2.2.4 Tags Per Question </h3>

In [ ]:
#storing the count of tag in each question in list 'tag_count'
tag_quest_count=tag_dtm.sum(axis=1).tolist()
#converting lists of list into a single list
tag_quest_count=[int(j) for i in tag_quest_count for j in i]
print('We have total {} datapoints'.format(len(tag_quest_count)))

In [ ]:
print( "Maximum number of tags per question: %d"%max(tag_quest_count))
print( "Minimum number of tags per question: %d"%min(tag_quest_count))
print( "Avg. number of tags per question: %f"% ((sum(tag_quest_count)*1.0)/len(tag_quest_count)))

In [ ]:
sns.countplot(x=tag_quest_count, palette='Set3')
plt.title('Number of tags in the question')
plt.xlabel('Number of tags')
plt.ylabel('Number of questions')
plt.show()

<b>Observations:</b><br />
1. Maximum number of tags per question: 5
2. Minimum number of tags per question: 1
3. Avg. number of tags per question: 2.899
4. Most of the questions are having 2 or 3 tags

<h3>2.2.5 Most Frequent Tags </h3>

In [ ]:
from wordcloud import WordCloud
#plotting wordcloud

start=datetime.now()

#converting result dictionary to list of tuples

tup=dict(result.items())

wordcloud=WordCloud(background_color='white',
                   width=1600, height=1000).generate_from_frequencies(tup)
fif=plt.figure(figsize=(30,20))
plt.imshow(wordcloud)
plt.axis('off')
plt.tight_layout(pad=0)
#fig.savefig('tag.png')
plt.show()
print('time taken: ', datetime.now()-start)

<b>Observations:</b><br />
A look at the word cloud shows that "c#", "java", "php", "asp.net", "javascript", "c++" are some of the most frequent tags.

<h3> 2.2.6 The top 20 tags </h3>

In [ ]:
i = np.arange(30)

# Plot the bar chart
top_30_tags = tag_df_sorted['Tags'].head(30)
top_30_counts = tag_df_sorted['Counts'].head(30)
plt.bar(i, top_30_counts)
plt.title('Frequency of top 30 Tags')
plt.xticks(i, top_30_tags, rotation=90) 
plt.xlabel('Tags')
plt.ylabel('Counts')

plt.show()

<b>Observations:</b><br />
1. Majority of the most frequent tags are programming language.
2. C# is the top most frequent programming language.
3. Android, IOS, Linux and windows are among the top most frequent operating systems.

<h3> 2.3 Cleaning and preprocessing of Questions </h3>

<h3> 2.3.1 Preprocessing </h3>

<ol> 
    <li> Sample 1M data points </li>
    <li> Separate out code-snippets from Body </li>
    <li> Remove Spcial characters from Question title and description (not in code)</li>
    <li> Remove stop words (Except 'C') </li>
    <li> Remove HTML Tags </li>
    <li> Convert all the characters into small letters </li>
    <li> Use SnowballStemmer to stem the words </li>
</ol>

In [ ]:
def striphtml(data):
    cleanr=re.compile('<.*?>')
    cleantext=re.sub(cleanr," ",str(data))
    return cleantext
stop_words=set(stopwords.words('english'))
stemmer=SnowballStemmer('english')

In [ ]:
def create_connection(db_file):
    try:
        conn=sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)
    return None

def create_table(conn,create_table_sql):
    try:
        c=conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)


def checkTableExists(dbcon):
    cursr=dbcon.cursor()
    str="SELECT name FROM sqlite_master WHERE type='table'"
    table_names=cursr.execute(str)
    print("tables in the database:")
    tables=table_names.fetchall()
    print(tables[0][0])
    return(len(tables))

def create_database_table(database,query):
    conn=create_connection(database)
    if conn is not None:
        create_table(conn,query)
        checkTableExists(conn)
    else:
        print("Error! cannot create database connection")
        conn.close()
        
sql_create_table="""CREATE TABLE IF NOT EXISTS QuestionsProcessed (question text NOT NULL, code text, tags text, words_pre integer, words_post integer, is_code integer);"""
create_database_table("Processed.db", sql_create_table)
    

In [ ]:
start=datetime.now()
read_db='train_no_dup.db'
write_db='Processed.db'

if os.path.isfile(read_db):
    conn_r=create_connection(read_db)
    if conn_r is not None:
        reader=conn_r.cursor()
        reader.execute("SELECT Title, Body, Tags \
        FROM no_dup_train \
        ORDER BY RANDOM() LIMIT 1000000;")
        
if os.path.isfile(write_db):
    conn_w=create_connection(write_db)
    if conn_w is not None:
        tables= checkTableExists(conn_w)
        writer=conn_w.cursor()
        if tables!=0:
            writer.execute("DELETE FROM QuestionsProcessed WHERE 1")
            print("cleared All the rows")
print("time taken :", datetime.now()-start)

__we create a new data base to store the sampled and preprocessed questions__

In [ ]:
start=datetime.now()
preprocessed_data_list=[]
reader.fetchone()
questions_with_code=0
len_pre=0
len_post=0
questions_processed=0

for row in reader:
    is_code=0
    title,question,tags=row[0],row[1],row[2]
    
    if '<code>' in question:
        questions_with_code+=1
        is_code=1
    x=len(question)+len(title)
    len_pre+=x
    
    code=str(re.findall(r'<code>(.*?)</code>',question,flags=re.DOTALL))
    
    question=re.sub('<code>(.*?)</code>','',question,flags=re.MULTILINE|re.DOTALL)
    question=striphtml(question.encode('utf-8'))
    
    question=str(title)+" "+str(question)
    question=re.sub(r'[^A-Za-z]+',' ',question)
    words=word_tokenize(str(question.lower()))
    #Removing all single letter and and stopwords from question exceptt for the letter 'c'
    question=' '.join(str(stemmer.stem(j)) for j in words if j not in stop_words and (len(j)!=1 or j=='c'))
    
    len_post+=len(question)
    tup=(question,code,tags,x,len(question),is_code)
    questions_processed+=1
    writer.execute("INSERT into QuestionsProcessed(question,code,tags,words_pre,words_post,is_code) values (?,?,?,?,?,?)",tup)
    if(questions_processed%100000==0):
        print('Number of questions processed =',questions_processed)
        
        
        
if questions_processed > 0:
    no_dup_avg_len_pre = len_pre / questions_processed
    no_dup_avg_len_post = len_post / questions_processed
    print("Avg. length of questions (Title+Body) before processing: %d" % no_dup_avg_len_pre)
    print("Avg. length of questions (Title+Body) after processing: %d" % no_dup_avg_len_post)
else:
    print("No questions processed, so cannot calculate averages.")

print("Time taken to run this cell :", datetime.now() - start)

In [ ]:
conn_r.commit()
conn_w.commit()
conn_r.close()
conn_w.close()

In [ ]:
if os.path.isfile(write_db):
    conn_r=create_connection(write_db)
    if conn_r is not None:
        reader=conn_r.cursor()
        reader.execute("SELECT question\
        FROM QuestionsProcessed\
        LIMIT 10")
        print("Questions after preprocessing")
        print('+'*100)
        reader.fetchone()
        for row in reader:
            print(row)
            print('-'*100)
conn_r.commit()
conn_r.close()

In [ ]:
#Taking 1M entries to a df
start=datetime.now()
write_db='Processed.db'
if os.path.isfile(write_db):
    conn_r=create_connection(write_db)
    if conn_r is not None:
        preprocessed_data=pd.read_sql_query("""SELECT question, Tags\
                                                FROM QuestionsProcessed""",conn_r)
        
conn_r.commit()
conn_r.close()
print('done : ',datetime.now()-start)

In [ ]:
preprocessed_data.head()

In [ ]:
print("number of data points in sample :", preprocessed_data.shape[0])
print("number of dimensions :", preprocessed_data.shape[1])